<table style="width: 100%; text-align: center; margin-left: auto; margin-right: auto;">
    <tr>
        <td><img src="https://lineasdebasepublicas.mma.gob.cl/images/mma_logo.png" alt="Ministerio del Medio Ambiente" style="width: 150px;"/></td>
        <td><img src="https://lineasdebasepublicas.mma.gob.cl/images/Logo.svg" alt="Proyecto LBP" style="width: 150px;"/></td>
    </tr>
</table>

## Notebook para acceder a los servicios de almacen de datos geoespaciales de la Plataforma Tecnológica Línea de Base Pública (PTLBP)

Consta de: 

1. **INFORMACIÓN PRIMARIA**
   - Levantamientos de información en terreno por región.
2. **INFORMACIÓN SECUNDARIA**
   - Levantamiento de información ya existente en diferentes entidades.

### Comentarios detallados del notebook
Este notebook tiene como objetivo acceder a los servicios de almacenamiento de datos geoespaciales de la Plataforma Tecnológica Línea de Base Pública (PTLBP). A continuación, se describen las funcionalidades implementadas:

1. **Importación de librerías**: Se importan las librerías necesarias para realizar solicitudes HTTP, manipular datos geoespaciales y trabajar con archivos.
2. **Configuración de pandas**: Se ajustan las opciones de visualización de pandas para mostrar todos los datos sin truncarlos.
3. **Listado de capas disponibles**: Se conecta al servicio WFS para obtener las capas disponibles y mostrarlas en formato HTML.
4. **Descarga de capas**: Se implementa una función para descargar capas en diferentes formatos geoespaciales y comprimirlas en archivos ZIP.

### Importar Librerias

In [ ]:
# Importar librerías necesarias
from pathlib import Path
import requests  
import urllib3  
import zipfile  
import pandas as pd  
import geopandas as gpd  
from xml.etree import ElementTree as ET  # Para analizar respuestas XML
from IPython.display import HTML, display  # Para mostrar contenido HTML en el notebook

# Deshabilitar advertencias de seguridad SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

### Configuraciones para listado de datos

In [ ]:
# Configurar opciones de pandas
# Estas configuraciones permiten mostrar todos los datos sin truncarlos
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas

### Video explicativo de funciones

In [ ]:
# ID del video de YouTube
video_id = "yyWbmsr3dRE"

# Código HTML para mostrar un enlace al video
enlace_html = f"""
<div style='text-align: center;'>
    <p>Si el video no se muestra correctamente, haz clic en el siguiente enlace:</p>
    <a href='https://www.youtube.com/watch?v={video_id}' target='_blank'>Ver video en YouTube</a>
</div>
"""

# Mostrar el enlace
display(HTML(enlace_html))

### Dirección de servicio Web Feature Services (WFS) Institucional

In [ ]:
wfs_url = 'https://lineasdebasepublicas.mma.gob.cl/geoserver/lbp/ows?service=WFS&version=1.1.0&request=GetCapabilities'

### Función para listar las capas del servicio WFS

In [ ]:
# Función optimizada para listar las capas disponibles en el servicio WFS
# Esta función realiza una solicitud al servicio WFS y analiza la respuesta XML para obtener las capas disponibles
# Parámetros:
# - url: URL del servicio WFS
# Retorna:
# - Una lista de tuplas con el nombre y título de cada capa

def listar_capas(url):
    try:
        # Realizar solicitud HTTP
        response = requests.get(url, verify=False, timeout=10)  # Agregar tiempo de espera para evitar bloqueos
        response.raise_for_status()  # Verificar que la solicitud fue exitosa

        # Analizar respuesta XML
        tree = ET.fromstring(response.content)
        namespaces = {
            'wfs': 'http://www.opengis.net/wfs',
            'ows': 'http://www.opengis.net/ows'
        }

        # Extraer capas utilizando comprensión de listas
        capas = [
            (
                feature.find('wfs:Name', namespaces).text,
                (feature.find('ows:Title', namespaces) or feature.find('wfs:Title', namespaces)).text
            )
            for feature in tree.findall('.//wfs:FeatureType', namespaces)
        ]

        # Filtrar capas con datos válidos
        capas = [(name, title) for name, title in capas if name and title]
        return capas

    except requests.exceptions.Timeout:
        print("La solicitud al servicio WFS excedió el tiempo de espera.")
        return []
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con el servicio WFS: {e}")
        return []
    except ET.ParseError:
        print("Error al analizar la respuesta XML del servicio WFS.")
        return []

### Funciones para mostrar catalogo de datos y descarga de datos 

In [ ]:
# Función para descargar capas seleccionadas en diferentes formatos
# Esta función permite descargar capas geoespaciales en formatos como GeoPackage, GeoJSON y Shapefile
# Parámetros:
# - selected_indices: Índices de las capas seleccionadas
# - download_format: Formato de descarga (GeoPackage, GeoJSON, Shapefile)

def download_layers(selected_indices, download_format):
    selected_layers = [capas[i] for i in selected_indices]  # Obtener capas seleccionadas

    # Función auxiliar para guardar capas en formato Shapefile
    def save_shapefile(gdf, filepath):
        geom_types = gdf.geometry.type.unique()  # Tipos de geometría en la capa
        supported_geom_types = ['Polygon', 'MultiPolygon', 'LineString', 'MultiLineString', 'Point', 'MultiPoint']
        for geom_type in supported_geom_types:
            if geom_type in geom_types:
                gdf[gdf.geometry.type == geom_type].to_file(filepath, driver='ESRI Shapefile')
                return
        print(f"Tipo de geometría no soportado: {geom_types}")

    # Mapear formatos a funciones de guardado
    format_functions = {
        'GeoPackage': lambda gdf, filepath: gdf.to_file(filepath, layer=file_name_base, driver="GPKG"),
        'GeoJSON': lambda gdf, filepath: gdf.to_file(filepath, driver="GeoJSON"),
        'Shapefile': save_shapefile
    }

    output_dir = Path('~/catalogo_geo_ptlbp/data').expanduser()  # Directorio de salida
    output_dir.mkdir(parents=True, exist_ok=True)  # Crear directorio si no existe

    for name, title in selected_layers:
        layer_url = f"https://lineasdebasepublicas.mma.gob.cl/geoserver/lbp/wfs?service=WFS&version=2.1.0&request=GetFeature&typeName={name}&outputFormat=application/json"
        try:
            response = requests.get(layer_url, verify=False)  # Realizar solicitud HTTP
            response.raise_for_status()  # Verificar que la solicitud fue exitosa
            data = response.json()  # Analizar respuesta JSON
            if 'features' not in data:
                print(f"La capa {title} no contiene datos válidos.")
                continue

            gdf = gpd.GeoDataFrame.from_features(data['features'])  # Convertir a GeoDataFrame
            if gdf.crs is None:
                gdf.set_crs(epsg=3857, inplace=True)  # Establecer CRS si no está definido
            gdf = gdf.to_crs(epsg=4326)  # Reproyectar a EPSG:4326

            file_name_base = name.replace('lbp:vista_', '').replace(':', '_')  # Generar nombre de archivo
            filepath = output_dir / f"{file_name_base}.{download_format.lower() if download_format != 'Shapefile' else 'shp'}"

            if download_format in format_functions:
                format_functions[download_format](gdf, filepath)  # Guardar capa en el formato seleccionado

            zip_filename = output_dir / f"{file_name_base}.zip"  # Nombre del archivo ZIP
            with zipfile.ZipFile(zip_filename, 'w') as zipf:
                if download_format in ['GeoPackage', 'GeoJSON']:
                    zipf.write(filepath, filepath.name)
                    filepath.unlink()  # Eliminar archivo original
                elif download_format == 'Shapefile':
                    for ext in ['shp', 'shx', 'dbf', 'prj', 'cpg']:
                        full_path = output_dir / f"{file_name_base}.{ext}"
                        if full_path.exists():
                            zipf.write(full_path, full_path.name)
                            full_path.unlink()  # Eliminar archivo original
            print(f"Capa {title} descargada y comprimida como {zip_filename}")
        except requests.exceptions.RequestException as e:
            print(f"Error al descargar la capa {title}: {e}")
        except ValueError:
            print(f"Error al procesar los datos de la capa {title}.")

### Listado de capas disponibles

In [ ]:
capas = listar_capas(wfs_url)
capas_html = """
<h3 style="font-size: 20px;">Capas disponibles:</h3>
<ul style="font-size: 18px;">
"""
for i, (name, title) in enumerate(capas):
    capas_html += f"<li>{i+1}. {title}</li>"
capas_html += "</ul>"
display(HTML(capas_html))

### Seleccione capas por número. 
### Si desea más de una capa escríbala separada de coma (",")

In [ ]:
selected_indices = input("Introduce los números de las capas a seleccionar, separados por comas: ")
selected_indices = [int(i) - 1 for i in selected_indices.split(",")]

### Seleccione el formato de descarga:
### (GeoPackage, GeoJSON, Shapefile)

In [ ]:
formats = {1: "GeoPackage", 2: "GeoJSON", 3: "Shapefile"}
formats_html = """
<h3 style="font-size: 20px;">Formatos disponibles:</h3>
<ul style="font-size: 18px;">
"""
for i, format_name in formats.items():
    formats_html += f"<li>{i}. {format_name}</li>"
formats_html += "</ul>"
display(HTML(formats_html))

download_format = int(input("Introduce el número del formato de descarga: "))

# Validar formato seleccionado
if download_format in formats:
    download_format_str = formats[download_format]
    # Ejecutar la descarga
    download_layers(selected_indices, download_format_str)
else:
    print("Formato no válido")